Author: Guy, Michael, Joshua
Summary: This is a comprenhensive simulation engine for ambulance simulation. It uses the solution from two-stage stochastic program and robust programs to simulate how ambulances would perform. The simulation mostly runs using a priority-queue of events with respect to time. Events include things like ambulance arrives to the scene of call, ambulance arrives to hospital, ect.

In [1]:
import Pkg #this line doesn't seem to run either way for me
Pkg.add("Pkg")
Pkg.add("Dates")

   Updating registry at `C:\Users\Owner\.julia\registries\General`
  Resolving package versions...
No Changes to `C:\Users\Owner\Documents\Austin\AmbulanceDeployment\AmbulanceDeployment.jl-legacy\Project.toml`
No Changes to `C:\Users\Owner\Documents\Austin\AmbulanceDeployment\AmbulanceDeployment.jl-legacy\Manifest.toml`
  Resolving package versions...
No Changes to `C:\Users\Owner\Documents\Austin\AmbulanceDeployment\AmbulanceDeployment.jl-legacy\Project.toml`
No Changes to `C:\Users\Owner\Documents\Austin\AmbulanceDeployment\AmbulanceDeployment.jl-legacy\Manifest.toml`


In [2]:
"""This activates packages, namely ambulance deployment"""

guypath = "/Users/guyfarmer/ambulance_connect/AmbulanceDeployement/AmbulanceDeployment.jl-legacy/"
michaelpath= "/home/mhilborn/Desktop/UT-Spring2021/SeniorLab/AmbulanceDeployement/AmbulanceDeployment.jl-legacy"
joshpath = "C:/Users/Owner/Documents/Austin/AmbulanceDeployment/AmbulanceDeployment.jl-legacy/"

currentpath = joshpath

cd(currentpath)
Pkg.activate(".")

using DataFrames, JLD, Distributions, CSV, Random, Plots,JSON, DelimitedFiles
import DataStructures: PriorityQueue, enqueue!, dequeue!
include(".//src//problem.jl")
include(".//src//model.jl")

using AmbulanceDeployment, Dates

 Activating environment at `C:\Users\Owner\Documents\Austin\AmbulanceDeployment\AmbulanceDeployment.jl-legacy\Project.toml`
┌ Info: Precompiling AmbulanceDeployment [d2c08a37-75f3-4934-bcbc-64feba624d07]
└ @ Base loading.jl:1278
┌ Warning: Package AmbulanceDeployment does not have Random in its dependencies:
│ - If you have AmbulanceDeployment checked out for development and have
│   added Random as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with AmbulanceDeployment
└ Loading Random into AmbulanceDeployment from project dependency, future warnings for AmbulanceDeployment are suppressed.


In [3]:
"""Helper functions for the simuation engine?"""
#=
Author : Ng Yeesian
Modified : Guy Farmer / Zander Tedjo / Will Worthington / Michael Hilborn
generates the closest dispatch model
Summary :
Input :
Output :
=#
mutable struct ClosestDispatch <: DispatchModel
    drivetime::DataFrame
    candidates::Vector{Vector{Int}}
    available::Vector{Int}
    assignment::Vector{Int} # which location the ambulance is assigned to
    ambulances::Vector{Vector{Int}} # list of ambulances assigned to each location
    status::Vector{Symbol} # the current status of the ambulance
        # possible statuses: :available, :responding, :atscene, :conveying, :returning
    fromtime::Vector{Int} # the time it started the new status
    hospital::Vector{Int}
end

#=
Summary :
Input :
Output : ClosestDispatch struct
=#
function ClosestDispatch(p::DeploymentProblem, drivetime::DataFrame,distribution::Vector{Int})
    candidates = Any[]
    I = 1:p.nlocations
    for region in 1:p.nregions
        push!(candidates, I[vec(p.coverage[region,:])])
    end
    #println("creating the dispatch problem :$distribution")

    assignment = zeros(Int, p.nambulances)
    ambulances = [Int[] for i in 1:p.nlocations]
    k = 1
    for i in eachindex(distribution), j in 1:distribution[i]
        assignment[k] = i
        push!(ambulances[i], k)
        k += 1
    end
    status = fill(:available, p.nambulances)
    fromtime = zeros(Int, p.nambulances)
    hospital = zeros(Int, p.nambulances)
    
    ClosestDispatch(drivetime, candidates, distribution, assignment, ambulances, status, fromtime, hospital)

end

function ClosestDispatch(p::DeploymentProblem, problem::DispatchProblem)
    stn_names = [Symbol("stn$(i)_min") for i in 1:size(p.coverage,2)]
    ClosestDispatch(p, problem.emergency_calls[:, stn_names], problem.available)
end

function update_ambulances!(model::ClosestDispatch, i::Int, delta::Int)
    model.available[i] += delta
    @assert model.available[i] >= 0
end

function closest_available(dispatch::ClosestDispatch, id::Int, problem::DispatchProblem)
    location = 0
    min_time = Inf
    for i in dispatch.candidates[problem.emergency_calls[id, :neighborhood]]
        if problem.available[i] > 0 && dispatch.drivetime[id, i] < min_time
            #println("apparently this station is available : $i and here is the amount of ambulances avalable: $(problem.available[i])")
            location = i
            min_time = dispatch.drivetime[id, i]
        end
    end
    location
end

function respond_to!(dispatch::ClosestDispatch, i::Int, t::Int)
    @assert length(dispatch.ambulances[i]) > 0 "$i: $(dispatch.ambulances[i])"
    amb = popfirst!(dispatch.ambulances[i])
    # @assert redeploy.hospital[amb] == 0
    @assert amb != 0
    @assert dispatch.status[amb] == :available "$amb: $(dispatch.status[amb])"
    dispatch.status[amb] = :responding
    #dispatch.fromtime[amb] = t
    amb
end

function arriveatscene!(dispatch::ClosestDispatch, amb::Int, t::Int)
    @assert dispatch.status[amb] == :responding "$amb: $(dispatch.status[amb])"
    @assert dispatch.hospital[amb] == 0
    dispatch.status[amb] = :atscene
    dispatch.fromtime[amb] = t
end

function arriveathospital!(dispatch::DispatchModel, amb::Int, hosp::Int, t::Int)
    @assert dispatch.status[amb] == :atscene "$amb: $(dispatch.status[amb])"
    @assert dispatch.hospital[amb] != 0
    dispatch.status[amb] = :atHospital
    dispatch.fromtime[amb] = t
    dispatch.hospital[amb] = hosp
end

function returning_to!(dispatch::DispatchModel, amb::Int, t::Int)
    @assert dispatch.status[amb] == :atHospital "$amb: $(dispatch.status[amb])"
    @assert dispatch.hospital[amb] != 0
    dispatch.status[amb] = :returning
    dispatch.fromtime[amb] = t
    dispatch.assignment[amb]
end

returning_to! (generic function with 1 method)

In [58]:
""" simulation parameters """
turnaround = Distributions.LogNormal(3.65, 0.3)
lambda = 0
local_path = ""
model_name = "Stochastic"
namb = 50
ncalls = 1000
#using AmbulanceDeployment

pwd()

cd(currentpath * "/test/")
pwd()

#using AmbulanceDeployment
using Dates

#include("..//src//dispatch/closestdispatch.jl")
include("..//src//problem.jl")
#include("..//src//evaluate.jl") #if you include, you need to include JuMP
#include("..//src//simulate.jl")

hospitals = CSV.File(string(local_path,"../test/austin-data/hospitals.csv")) |> DataFrame
stations = CSV.File(string(local_path,"../test/austin-data/stations.csv")) |> DataFrame
hourly_calls = CSV.File("../test/austin-data/Full_WeekdayCalls.csv") |> DataFrame
adjacent_nbhd = CSV.File("../test/austin-data/adjacent_nbhd.csv") |> DataFrame
coverage = CSV.read("../test/austin-data/coverage_real.csv", DataFrame, header=false)
coverage = convert(Array{Bool, 2}, coverage[:, :])
incidents = CSV.File("../test/austin-data/austin_incidents.csv") |> DataFrame
solverstats = JLD.load("../src/outputs/austin_team_stats_1_03.jld")
amb_deployment = solverstats["amb_deployment"]
model_dict = Dict{String, Symbol}("Stochastic"=>:Stochastic, "Robust01"=>:Robust01, "Robust005"=>:Robust005, "Robust001"=>:Robust001, "Robust0001"=>:Robust0001,
          "Robust00001"=>:Robust00001, "MEXCLP"=>:MEXCLP, "MALP"=>:MALP)


p = DeploymentProblem(
          hourly_calls,
          adjacent_nbhd,
          coverage,
          namb = namb,
          train_filter = (hourly_calls[!,:year] .== 2019) .* (hourly_calls[!,:month] .<= 3)
          )
test_calls = CSV.File("../test/austin-data/austin_test_calls_update1.csv")|> DataFrame 
#update 1: remove negative indices
test_calls = test_calls[1:ncalls,:]
println("running $namb ambuances & $ncalls calls")

# distribution of ambulances across all of the stations
x = amb_deployment[model_dict[model_name]][namb]

#establishes the dispatch problem
problem = DispatchProblem(test_calls, hospitals, stations, p.coverage, x, turnaround=turnaround)

#establishes the dispatch model
dispatch = ClosestDispatch(p, problem)

Random.seed!(1234)



running 50 ambuances & 1000 calls


MersenneTwister(UInt32[0x000004d2], Random.DSFMT.DSFMT_state(Int32[-1393240018, 1073611148, 45497681, 1072875908, 436273599, 1073674613, -2043716458, 1073445557, -254908435, 1072827086  …  -599655111, 1073144102, 367655457, 1072985259, -1278750689, 1018350124, -597141475, 249849711, 382, 0]), [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], UInt128[0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000  …  0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x000000000000

In [60]:
i = 1
start_ind = (i-1)*1000+1
end_ind = (i)*1000
println("running $namb ambuances & $start_ind to $end_ind calls")
test_calls_batch = test_calls[start_ind:end_ind,:]

running 50 ambuances & 1 to 1000 calls


,interarrival_seconds,neighborhood,dow,stn1_min,stn2_min,stn3_min,stn4_min
,Int64,Int64,String,Float64,Float64,Float64,Float64
1,38,83,Wed,5.0,26.9957,10.791,13.1813
2,537,38,Wed,16.873,26.3413,22.8293,22.9943
3,98,129,Wed,21.5312,38.0153,22.2628,18.438
4,18,107,Wed,44.2525,37.6413,36.6008,50.837
5,475,113,Wed,14.653,29.5155,15.3847,13.4282
6,21,51,Wed,21.06,13.8817,18.8858,31.445
7,263,76,Wed,43.5517,34.0775,36.4807,52.2222
8,128,67,Wed,14.8368,17.8243,9.6105,23.073
9,62,52,Wed,17.9115,22.8192,17.5257,27.8428


In [61]:
test_calls = CSV.File("../test/austin-data/austin_test_calls_update1.csv")|> DataFrame 
#update 1: remove negative indices
test_calls = test_calls[1:ncalls,:]

,interarrival_seconds,neighborhood,dow,stn1_min,stn2_min,stn3_min,stn4_min
,Int64,Int64,String,Float64,Float64,Float64,Float64
1,38,83,Wed,5.0,26.9957,10.791,13.1813
2,537,38,Wed,16.873,26.3413,22.8293,22.9943
3,98,129,Wed,21.5312,38.0153,22.2628,18.438
4,18,107,Wed,44.2525,37.6413,36.6008,50.837
5,475,113,Wed,14.653,29.5155,15.3847,13.4282
6,21,51,Wed,21.06,13.8817,18.8858,31.445
7,263,76,Wed,43.5517,34.0775,36.4807,52.2222
8,128,67,Wed,14.8368,17.8243,9.6105,23.073
9,62,52,Wed,17.9115,22.8192,17.5257,27.8428


In [20]:
"""Simulation Enginer"""

Pkg.add("Query")
using Query

struct EMSEngine{T}
    eventlog::DataFrame
    eventqueue::PriorityQueue{T,Int,Base.Order.ForwardOrdering}
    shortqueue::PriorityQueue{T,Int,Base.Order.ForwardOrdering}
    guiArray::Array{Any, 1}
end

struct gui_event
    #call made / call responded / ambulance arrived / shortfall
    event_type::String
    #neighborhood where call is made from
    neighborhood_id::Int
    # station where call is made from
    deployment_id::Int
    # id of the event generated
    event_id::Int
    ambulance_id::Int
    arrival_time::Int
    remaining_amb::Int
    # current timestamp of event
    timestamp::DateTime
end

## generates the priority queue and instantiates the EMSEngine struct
function generateCalls(problem::DispatchProblem)
    ncalls = nrow(problem.emergency_calls)
    eventlog = DataFrame(
        id = 1:ncalls,
        dispatch_from = zeros(Int, ncalls),
        waittime = fill(0.0, ncalls),
        responsetime = fill(Inf, ncalls),
        scenetime = fill(Inf, ncalls),
        conveytime = fill(Inf, ncalls),
        returntime = fill(Inf, ncalls),
        return_to = zeros(Int, ncalls),
        return_type = fill(:station, ncalls),
        hospital = zeros(Int, ncalls),
        ambulance = zeros(Int, ncalls)
    )
    eventqueue = PriorityQueue{Tuple{Symbol,Int,Int,Int},Int}()
    shortqueue = PriorityQueue{Tuple{Symbol,Int,Int,Int},Int}()
    guiArray = Any[]
    for i in 1:nrow(problem.emergency_calls)
        t = problem.emergency_calls[i, :arrival_seconds]
        enqueue!(eventqueue, (:call, i, t, problem.emergency_calls[i, :neighborhood]), t)
    end
    EMSEngine{Tuple{Symbol,Int,Int,Int}}(eventlog, eventqueue, shortqueue, guiArray) ## returns the ems engine struct
end

function simulate_events!(problem::DispatchProblem, dispatch::ClosestDispatch)
    engine = generateCalls(problem)
    while(~isempty(engine.eventqueue))
        (event, id, t, value) = dequeue!(engine.eventqueue)

        if event == :call
            # if the event is reachable then change event to station2call
            # if not it is a shortfall
            println("calling event id: $id time: $t value: $value")
            call_event!(engine, problem, dispatch, id, t, value)
           
        elseif event == :station2call
            #once the ambulance arrives at the station change event to call2hospital
            println("Ambulance has arrived on the scene for event $id at time $t")
            station2call_event!(engine, problem, dispatch, id, t, value)
        elseif event == :call2hospital
            #once the ambulance arrives at the hospital it will hand over the patient and then 
            #change the deploy back to its station
            println("Ambulance for event $id has arrived at the hospital at time $t")
            call2hosp_event!(engine, problem, dispatch, id, t, value)

        elseif event == :hospital2station
            #once the ambulance drops off the patient it will reroute to the station
            println("Ambulance for event $id is returning to the station at $t")
            return_event!(engine, problem, dispatch, id, t, value)
            
        elseif event == :done
            #increase the number of ambulances at that station now that it is done
            done_event!(engine, problem, dispatch, id, t, value)
            
        end
    end
    println("This is the Short Queue")
    while(~isempty(engine.shortqueue))
       (event, id, t, value) = dequeue!(engine.shortqueue)
        print("$event, $id, $t, $value")
        print("This is not empty")
    end
    engine.eventlog , engine.guiArray
end


function call_event!(
        ems::EMSEngine,
        problem::DispatchProblem,
        dispatch::DispatchModel,
        id::Int, # the id of the emergency call
        t::Int, # the time of the emergency call
        nbhd::Int; # the neighborhood the call is from
    )

    #check if there is an ambulance within the coverage matrix
    if sum(problem.deployment[problem.coverage[nbhd,:]]) == 0
        @assert false "$id: no ambulance reachable for call at $nbhd"
    #check if one of the ambulances within the coverage is available
    elseif sum(problem.available[problem.coverage[nbhd,:]]) > 0
        i = closest_available(dispatch, id, problem)
        #println("closest available station : $i")
        @assert i > 0 "$id: dispatch from $i to nbhd $nbhd" # assume valid i (enforced by <if> condition)
        update_ambulances!(dispatch, i, -1)
        ems.eventlog[id, :dispatch_from] = i
        @assert problem.available[i] >= 0
        

        travel_time = ceil(Int, 60*problem.emergency_calls[id, Symbol("stn$(i)_min")])
        @assert travel_time >= 0
        if(t != problem.emergency_calls[id, :arrival_seconds])
            print("This is a shortfall")
        end
        ems.eventlog[id, :responsetime] = (travel_time + t - problem.emergency_calls[id,:arrival_seconds]) / 60 # minutes

        amb = respond_to!(dispatch, i, t)
        ems.eventlog[id, :ambulance] = amb

        event = gui_event("call responded", problem.emergency_calls[id, :neighborhood], i, id, amb,-1, problem.available[i] ,Dates.now())
        num_ambulances_array = Array{Integer}(undef, size(problem.available, 1))
        for i in 1:size(problem.available, 1)
            num_ambulances_array[i] = problem.available[i]
        end
        push!(ems.guiArray,event)
        push!(ems.guiArray,num_ambulances_array)
        enqueue!(ems.eventqueue, (:station2call, id, t + travel_time, amb), t + travel_time)
    #else queue it
    else
        # println(id, ": call from ", nbhd, " queued behind ", problem.wait_queue[nbhd])
        # event = gui_event("call made", nbhd, -1, id, -1,-1,-1,Dates.now())
        # push!(ems.guiArray,event)
        # problem.shortfalls = problem.shortfalls + 1
        # #push!(problem.shortfalls) # count shortfalls
        # push!(problem.wait_queue[nbhd], id) # queue the emergency call
        event = gui_event("shortfall", problem.emergency_calls[id, :neighborhood], -1, id, -1,-1,-1 ,Dates.now())
        push!(ems.guiArray, event)
        enqueue!(ems.shortqueue, (:call, id, t, problem.emergency_calls[id, :neighborhood]), t)
        println(id, ": call from ", nbhd, " is a shortfall")
    end
end

function station2call_event!(
        ems::EMSEngine,
        problem::DispatchProblem,
        dispatch::DispatchModel,
        id::Int, # the id of the emergency call
        t::Int, # the time of the emergency call
        amb::Int
    )
    arriveatscene!(dispatch, amb, t)
    # time the ambulance spends at the scene
    scene_time = ceil(Int,60*0.4*rand(problem.turnaround)) # 60sec*0.4*mean(40minutes) ~ 15minutes
    ems.eventlog[id, :scenetime] = scene_time / 60 # minutes
    @assert scene_time > 0
    
    event = gui_event("ambulance arrived", problem.emergency_calls[id, :neighborhood], -1, id, amb, t,-1 ,Dates.now())
    push!(ems.guiArray, event)
    enqueue!(ems.eventqueue, (:call2hospital, id, t + scene_time, amb), t + scene_time)
end

function call2hosp_event!(
        ems::EMSEngine,
        problem::DispatchProblem,
        dispatch::DispatchModel,
        id::Int, # the id of the emergency call
        t::Int, # the time of the emergency call
        amb::Int
    )
    h = let mintime = Inf, minindex = 0
        for h in 1:nrow(problem.hospitals)
            traveltime = problem.emergency_calls[id, Symbol("hosp$(h)_min")]
            if !isna(traveltime) && !(traveltime=="NA") && traveltime < mintime
                @assert traveltime >= 0
                minindex = h; mintime = traveltime
            end
        end
        minindex
    end
    @assert h != 0
    dispatch.hospital[amb] = ems.eventlog[id, :hospital] = h
    conveytime = 60*15 + ceil(Int, 60*problem.emergency_calls[id, Symbol("hosp$(h)_min")]) # ~20minutes
    ems.eventlog[id, :conveytime] = conveytime / 60 # minutes
    @assert conveytime >= 0 conveytime
    arriveathospital!(dispatch, amb, h, t)
    
    #         #call made / call responded / ambulance arrived / shortfall
#     event_type::String
#     #neighborhood where call is made from
#     neighborhood_id::Int
#     # station where call is made from
#     deployment_id::Int
#     # id of the event generated
#     event_id::Int
#     ambulance_id::Int
#     arrival_time::Int
#     remaining_amb::Int
#     # current timestamp of event
#     timestamp::DateTime
    event = gui_event("ambulance hospital", problem.emergency_calls[id, :neighborhood], -1, id, amb, t,-1 ,Dates.now())
    push!(ems.guiArray, event)
    
    
    enqueue!(ems.eventqueue, (:hospital2station, id, t+conveytime, amb), t+conveytime)
end

function return_event!(
        ems::EMSEngine,
        problem::DispatchProblem,
        dispatch::DispatchModel,
        id::Int,
        t::Int,
        amb::Int
    )
    stn = returning_to!(dispatch, amb, t)
    h = dispatch.hospital[amb]
    returntime = ceil(Int,60*problem.hospitals[h, Symbol("stn$(stn)_min")]) # ~ 10minutes
    dispatch.hospital[amb] = 0
    ems.eventlog[id, :returntime] = returntime / 60 # minutes
    @assert returntime >= 0 returntime
    t_end = t + returntime
    
#         #call made / call responded / ambulance arrived / shortfall
#     event_type::String
#     #neighborhood where call is made from
#     neighborhood_id::Int
#     # station where call is made from
#     deployment_id::Int
#     # id of the event generated
#     event_id::Int
#     ambulance_id::Int
#     arrival_time::Int
#     remaining_amb::Int
#     # current timestamp of event
#     timestamp::DateTime
    
    event = gui_event("ambulance returning", problem.emergency_calls[id, :neighborhood], -1, id, amb, t,-1 ,Dates.now())
    push!(ems.guiArray, event)
    
    
    enqueue!(ems.eventqueue, (:done, id, t_end, amb), t_end)
    # num_ambulances_array = Array{Integer}(undef, size(problem.available, 1))
    # for i in 1:size(problem.available, 1)
    #     num_ambulances_array[i] = problem.available[i]
    # end
    # push!(ems.guiArray,num_ambulances_array)
end

function done_event!(
        ems::EMSEngine,
        problem::DispatchProblem,
        dispatch::DispatchModel,
        id::Int,
        t::Int,
        amb::Int
    )
    i = dispatch.assignment[amb]
    println("Ambulance from event $id has returned to station $i")
    @assert i > 0 "$id: dispatch from $i to nbhd $nbhd" # assume valid i (enforced by <if> condition)
    update_ambulances!(dispatch, i, 1)
    push!(dispatch.ambulances[i], amb)
    dispatch.status[amb] = :available
    ems.eventlog[id, :return_to] = id
    @assert problem.available[i] > 0
    
#         #call made / call responded / ambulance arrived / shortfall
#     event_type::String
#     #neighborhood where call is made from
#     neighborhood_id::Int
#     # station where call is made from
#     deployment_id::Int
#     # id of the event generated
#     event_id::Int
#     ambulance_id::Int
#     arrival_time::Int
#     remaining_amb::Int
#     # current timestamp of event
#     timestamp::DateTime
    
    event = gui_event("ambulance returned", problem.emergency_calls[id, :neighborhood], -1, id, amb, t,-1 ,Dates.now())
    push!(ems.guiArray, event)
    
    while(~isempty(ems.shortqueue))
        # record time difference
        (event, short_id, short_t, value) = dequeue!(ems.shortqueue)
        enqueue!(ems.eventqueue, (event, short_id, t , value), short_t)
    end
end


  Resolving package versions...
No Changes to `C:\Users\Owner\Documents\Austin\AmbulanceDeployment\AmbulanceDeployment.jl-legacy\Project.toml`
No Changes to `C:\Users\Owner\Documents\Austin\AmbulanceDeployment\AmbulanceDeployment.jl-legacy\Manifest.toml`


done_event! (generic function with 1 method)

In [53]:
"""call simulation"""
#@time df, guiArray = simulate_events!(problem, dispatch)
df, guiArray = simulate_events!(problem, dispatch)

calling event id: 1 time: 38 value: 83
Ambulance has arrived on the scene for event 1 at time 338
calling event id: 2 time: 575 value: 38
calling event id: 3 time: 673 value: 129
calling event id: 4 time: 691 value: 107
Ambulance has arrived on the scene for event 3 at time 973
calling event id: 5 time: 1166 value: 113
calling event id: 6 time: 1187 value: 51
Ambulance has arrived on the scene for event 2 at time 1250
Ambulance has arrived on the scene for event 4 at time 1302
calling event id: 7 time: 1450 value: 76
Ambulance has arrived on the scene for event 5 at time 1466
Ambulance has arrived on the scene for event 6 at time 1487
Ambulance for event 1 has arrived at the hospital at time 1536
calling event id: 8 time: 1578 value: 67
calling event id: 9 time: 1640 value: 52
Ambulance for event 3 has arrived at the hospital at time 1678
calling event id: 10 time: 1838 value: 67
Ambulance has arrived on the scene for event 8 at time 1878
calling event id: 11 time: 1899 value: 67
calli

Ambulance has arrived on the scene for event 35 at time 9813
Ambulance has arrived on the scene for event 36 at time 9950
Ambulance has arrived on the scene for event 34 at time 9969
calling event id: 37 time: 10091 value: 98
Ambulance for event 31 is returning to the station at 10320
Ambulance for event 32 has arrived at the hospital at time 10379
Ambulance for event 27 is returning to the station at 10427
Ambulance for event 33 has arrived at the hospital at time 10563
calling event id: 38 time: 10595 value: 98
Ambulance for event 29 is returning to the station at 10609
Ambulance from event 27 has returned to station 17
Ambulance for event 26 is returning to the station at 10676
Ambulance from event 21 has returned to station 43
Ambulance has arrived on the scene for event 37 at time 10699
Ambulance for event 28 is returning to the station at 10721
Ambulance for event 35 has arrived at the hospital at time 10727
calling event id: 39 time: 10727 value: 52
calling event id: 40 time: 10

calling event id: 63 time: 19271 value: 98
Ambulance has arrived on the scene for event 62 at time 19313
Ambulance for event 61 has arrived at the hospital at time 19407
calling event id: 64 time: 19691 value: 51
Ambulance for event 55 is returning to the station at 19703
Ambulance has arrived on the scene for event 63 at time 19877
Ambulance from event 55 has returned to station 17
calling event id: 65 time: 19963 value: 98
Ambulance has arrived on the scene for event 64 at time 19991
Ambulance for event 62 has arrived at the hospital at time 20000
Ambulance for event 60 is returning to the station at 20039
Ambulance for event 59 is returning to the station at 20315
calling event id: 66 time: 20471 value: 113
Ambulance from event 60 has returned to station 11
Ambulance has arrived on the scene for event 65 at time 20686
Ambulance for event 64 has arrived at the hospital at time 20781
calling event id: 67 time: 20814 value: 36
Ambulance from event 59 has returned to station 1
Ambulance

Ambulance for event 87 is returning to the station at 31257
Ambulance for event 88 is returning to the station at 31293
Ambulance for event 91 has arrived at the hospital at time 31434
Ambulance for event 92 has arrived at the hospital at time 31487
Ambulance from event 88 has returned to station 8
Ambulance for event 89 is returning to the station at 31738
Ambulance from event 87 has returned to station 2
Ambulance from event 85 has returned to station 22
Ambulance from event 89 has returned to station 25
calling event id: 93 time: 32356 value: 66
Ambulance for event 91 is returning to the station at 32634
Ambulance for event 92 is returning to the station at 32687
Ambulance from event 91 has returned to station 8
calling event id: 94 time: 32957 value: 66
Ambulance for event 90 is returning to the station at 32965
Ambulance has arrived on the scene for event 93 at time 33076
Ambulance from event 92 has returned to station 14
Ambulance from event 90 has returned to station 39
Ambulanc

Ambulance has arrived on the scene for event 122 at time 45307
Ambulance from event 111 has returned to station 43
Ambulance for event 119 has arrived at the hospital at time 45372
Ambulance has arrived on the scene for event 123 at time 45401
calling event id: 126 time: 45420 value: 143
Ambulance has arrived on the scene for event 124 at time 45577
Ambulance for event 120 has arrived at the hospital at time 45779
Ambulance for event 117 has arrived at the hospital at time 45779
calling event id: 127 time: 45780 value: 97
Ambulance has arrived on the scene for event 126 at time 45976
Ambulance for event 116 is returning to the station at 46084
calling event id: 128 time: 46110 value: 82
Ambulance has arrived on the scene for event 127 at time 46154
Ambulance for event 121 has arrived at the hospital at time 46279
Ambulance has arrived on the scene for event 125 at time 46289
Ambulance for event 122 has arrived at the hospital at time 46299
Ambulance has arrived on the scene for event 1

...

calling event id: 898 time: 252637 value: 129
Ambulance for event 891 has arrived at the hospital at time 252673
Ambulance for event 893 has arrived at the hospital at time 252690
Ambulance has arrived on the scene for event 894 at time 252760
calling event id: 899 time: 252799 value: 68
calling event id: 900 time: 252837 value: 71
Ambulance for event 889 is returning to the station at 252846
calling event id: 901 time: 252877 value: 129
Ambulance from event 886 has returned to station 28
Ambulance has arrived on the scene for event 897 at time 252920
Ambulance has arrived on the scene for event 898 at time 252937
Ambulance for event 884 is returning to the station at 253018
Ambulance has arrived on the scene for event 899 at time 253099
calling event id: 902 time: 253137 value: 67
Ambulance from event 884 has returned to station 8
calling event id: 903 time: 253176 value: 83
calling event id: 904 time: 253220 value: 82
Ambulance has arrived on the scene for event 896 at time 253288
Am

calling event id: 928 time: 259399 value: 82
Ambulance has arrived on the scene for event 923 at time 259492
Ambulance for event 918 is returning to the station at 259509
Ambulance has arrived on the scene for event 927 at time 259509
calling event id: 929 time: 259529 value: 116
Ambulance from event 910 has returned to station 2
Ambulance has arrived on the scene for event 925 at time 259575
Ambulance for event 919 has arrived at the hospital at time 259635
Ambulance from event 917 has returned to station 19
Ambulance has arrived on the scene for event 928 at time 259699
Ambulance from event 913 has returned to station 15
Ambulance for event 921 has arrived at the hospital at time 259771
Ambulance has arrived on the scene for event 929 at time 259829
Ambulance from event 911 has returned to station 2
Ambulance has arrived on the scene for event 926 at time 259884
calling event id: 930 time: 259902 value: 113
Ambulance from event 918 has returned to station 11
Ambulance from event 900 

Ambulance for event 937 is returning to the station at 264830
Ambulance for event 941 is returning to the station at 264963
Ambulance for event 948 is returning to the station at 264978
Ambulance for event 945 is returning to the station at 264979
Ambulance from event 939 has returned to station 28
Ambulance from event 940 has returned to station 1
Ambulance from event 943 has returned to station 15
Ambulance from event 948 has returned to station 8
Ambulance for event 952 has arrived at the hospital at time 265172
Ambulance for event 942 is returning to the station at 265271
calling event id: 958 time: 265275 value: 51
Ambulance from event 937 has returned to station 25
Ambulance for event 944 is returning to the station at 265300
Ambulance from event 941 has returned to station 11
Ambulance for event 946 is returning to the station at 265454
Ambulance has arrived on the scene for event 955 at time 265458
Ambulance has arrived on the scene for event 957 at time 265490
Ambulance from e

Ambulance has arrived on the scene for event 980 at time 273780
Ambulance has arrived on the scene for event 982 at time 273824
Ambulance has arrived on the scene for event 979 at time 273878
Ambulance has arrived on the scene for event 978 at time 273941
Ambulance for event 975 has arrived at the hospital at time 273959
Ambulance has arrived on the scene for event 981 at time 274144
Ambulance from event 974 has returned to station 1
Ambulance for event 977 has arrived at the hospital at time 274340
calling event id: 983 time: 274726 value: 98
Ambulance for event 978 has arrived at the hospital at time 274743
Ambulance for event 979 has arrived at the hospital at time 274784
Ambulance for event 980 has arrived at the hospital at time 275005
Ambulance for event 982 has arrived at the hospital at time 275053
Ambulance for event 975 is returning to the station at 275159
Ambulance for event 981 has arrived at the hospital at time 275291
Ambulance has arrived on the scene for event 983 at t

(1000×11 DataFrame. Omitted printing of 6 columns
│ Row  │ id    │ dispatch_from │ waittime │ responsetime │ scenetime │
│      │ Int64 │ Int64         │ Float64  │ Float64      │ Float64   │
├──────┼───────┼───────────────┼──────────┼──────────────┼───────────┤
│ 1    │ 1     │ 1             │ 0.0      │ 5.0          │ 19.9667   │
│ 2    │ 2     │ 43            │ 0.0      │ 11.25        │ 13.2833   │
│ 3    │ 3     │ 36            │ 0.0      │ 5.0          │ 11.75     │
│ 4    │ 4     │ 34            │ 0.0      │ 10.1833      │ 11.75     │
│ 5    │ 5     │ 14            │ 0.0      │ 5.0          │ 19.95     │
│ 6    │ 6     │ 22            │ 0.0      │ 5.0          │ 29.8833   │
│ 7    │ 7     │ 39            │ 0.0      │ 24.55        │ 26.8333   │
│ 8    │ 8     │ 8             │ 0.0      │ 5.0          │ 18.0667   │
│ 9    │ 9     │ 8             │ 0.0      │ 11.35        │ 13.1833   │
│ 10   │ 10    │ 8             │ 0.0      │ 5.0          │ 14.2      │
⋮
│ 990  │ 990   │ 44      

In [83]:
"""batch call simulation"""
batches = 12
test_calls = CSV.File("../test/austin-data/austin_test_calls_update1.csv")|> DataFrame 
#update 1: remove negative indices
using DelimitedFiles

for i = 6:12
    """ simulation parameters """
    p = DeploymentProblem(
              hourly_calls,
              adjacent_nbhd,
              coverage,
              namb = namb,
              train_filter = (hourly_calls[!,:year] .== 2019) .* (hourly_calls[!,:month] .<= 3)
              )
    test_calls = CSV.File("../test/austin-data/austin_test_calls_update1.csv")|> DataFrame 
    #update 1: remove negative indices
    start_ind = (i-1)*1000+1
    end_ind = (i)*1000
    println("running $namb ambuances & $start_ind to $end_ind calls")
    test_calls_batch = test_calls[start_ind:end_ind,:]

    # distribution of ambulances across all of the stations
    x = amb_deployment[model_dict[model_name]][namb]

    #establishes the dispatch problem
    problem = DispatchProblem(test_calls_batch, hospitals, stations, p.coverage, x, turnaround=turnaround)

    #establishes the dispatch model
    dispatch = ClosestDispatch(p, problem)

    #@time df, guiArray = simulate_events!(problem, dispatch)
    df, guiArray = simulate_events!(problem, dispatch)
    print(mean(df["responsetime"]))
    
    filename = string(model_name,string(namb),"calls",string(ncalls),"batch",string(i),".csv")
    open(filename, "w") do filename
        writedlm(filename,  df["responsetime"])
    end
    print(filename)
end

running 50 ambuances & 5001 to 6000 calls
calling event id: 1 time: 231 value: 82
calling event id: 2 time: 526 value: 83
Ambulance has arrived on the scene for event 1 at time 531
Ambulance has arrived on the scene for event 2 at time 826
Ambulance for event 1 has arrived at the hospital at time 1212
Ambulance for event 2 has arrived at the hospital at time 1830
calling event id: 3 time: 1904 value: 69
Ambulance has arrived on the scene for event 3 at time 2409
Ambulance for event 1 is returning to the station at 2412
calling event id: 4 time: 2820 value: 98
Ambulance from event 1 has returned to station 3
calling event id: 5 time: 3026 value: 98
calling event id: 6 time: 3275 value: 83
Ambulance for event 2 is returning to the station at 3334
calling event id: 7 time: 3394 value: 93
Ambulance has arrived on the scene for event 4 at time 3426
Ambulance for event 3 has arrived at the hospital at time 3475
Ambulance has arrived on the scene for event 5 at time 3634
calling event id: 8 t

calling event id: 39 time: 11299 value: 67
calling event id: 40 time: 11390 value: 98
calling event id: 41 time: 11411 value: 156
Ambulance for event 28 is returning to the station at 11424
Ambulance for event 34 has arrived at the hospital at time 11426
calling event id: 42 time: 11452 value: 98
Ambulance for event 26 is returning to the station at 11486
calling event id: 43 time: 11507 value: 82
Ambulance for event 27 has arrived at the hospital at time 11550
Ambulance for event 30 is returning to the station at 11551
Ambulance for event 32 has arrived at the hospital at time 11594
Ambulance has arrived on the scene for event 38 at time 11597
Ambulance has arrived on the scene for event 39 at time 11599
Ambulance for event 33 has arrived at the hospital at time 11819
calling event id: 44 time: 11826 value: 98
calling event id: 45 time: 11890 value: 82
Ambulance has arrived on the scene for event 37 at time 11930
Ambulance has arrived on the scene for event 41 at time 11936
Ambulance 

Ambulance from event 42 has returned to station 3
calling event id: 58 time: 14986 value: 116
58: call from 116 is a shortfall
Ambulance for event 56 has arrived at the hospital at time 15019
Ambulance for event 62 has arrived at the hospital at time 15038
calling event id: 70 time: 15046 value: 156
Ambulance from event 33 has returned to station 30
calling event id: 58 time: 15059 value: 116
This is a shortfallcalling event id: 71 time: 15068 value: 115
calling event id: 72 time: 15140 value: 99
72: call from 99 is a shortfall
calling event id: 73 time: 15143 value: 82
calling event id: 74 time: 15152 value: 67
Ambulance from event 40 has returned to station 1
calling event id: 72 time: 15196 value: 99
This is a shortfallAmbulance from event 44 has returned to station 17
Ambulance for event 47 is returning to the station at 15262
Ambulance for event 49 is returning to the station at 15283
Ambulance from event 43 has returned to station 28
Ambulance for event 46 is returning to the sta

83: call from 116 is a shortfall
calling event id: 84 time: 18071 value: 116
84: call from 116 is a shortfall
Ambulance for event 80 has arrived at the hospital at time 18105
Ambulance has arrived on the scene for event 86 at time 18134
Ambulance from event 68 has returned to station 8
calling event id: 83 time: 18149 value: 116
83: call from 116 is a shortfall
calling event id: 84 time: 18149 value: 116
84: call from 116 is a shortfall
Ambulance for event 81 has arrived at the hospital at time 18159
Ambulance for event 74 is returning to the station at 18210
Ambulance from event 64 has returned to station 43
calling event id: 83 time: 18237 value: 116
83: call from 116 is a shortfall
calling event id: 84 time: 18237 value: 116
84: call from 116 is a shortfall
Ambulance for event 70 is returning to the station at 18273
Ambulance has arrived on the scene for event 87 at time 18321
Ambulance has arrived on the scene for event 88 at time 18359
Ambulance for event 85 has arrived at the hos

Ambulance has arrived on the scene for event 109 at time 22108
Ambulance has arrived on the scene for event 105 at time 22111
calling event id: 114 time: 22115 value: 113
Ambulance for event 89 is returning to the station at 22168
Ambulance has arrived on the scene for event 111 at time 22282
Ambulance for event 101 has arrived at the hospital at time 22288
calling event id: 115 time: 22320 value: 109
Ambulance for event 104 has arrived at the hospital at time 22341
calling event id: 116 time: 22380 value: 82
Ambulance has arrived on the scene for event 113 at time 22464
Ambulance from event 89 has returned to station 39
calling event id: 112 time: 22493 value: 116
112: call from 116 is a shortfall
Ambulance for event 97 is returning to the station at 22565
calling event id: 117 time: 22576 value: 37
calling event id: 118 time: 22580 value: 113
calling event id: 119 time: 22582 value: 113
119: call from 113 is a shortfall
Ambulance from event 94 has returned to station 1
calling event 

...

Ambulance for event 573 has arrived at the hospital at time 142971
Ambulance for event 574 has arrived at the hospital at time 142971
Ambulance for event 569 is returning to the station at 142995
Ambulance from event 567 has returned to station 8
calling event id: 582 time: 143022 value: 67
Ambulance from event 566 has returned to station 14
Ambulance for event 572 has arrived at the hospital at time 143083
Ambulance from event 569 has returned to station 13
Ambulance for event 570 is returning to the station at 143187
Ambulance has arrived on the scene for event 580 at time 143215
Ambulance from event 568 has returned to station 3
Ambulance for event 579 has arrived at the hospital at time 143274
Ambulance from event 564 has returned to station 4
Ambulance has arrived on the scene for event 582 at time 143322
Ambulance from event 570 has returned to station 8
Ambulance for event 578 has arrived at the hospital at time 143386
calling event id: 583 time: 143515 value: 67
calling event i

calling event id: 614 time: 148808 value: 157
Ambulance has arrived on the scene for event 608 at time 148821
calling event id: 615 time: 148850 value: 157
calling event id: 616 time: 148860 value: 83
Ambulance from event 594 has returned to station 22
Ambulance for event 603 has arrived at the hospital at time 148891
Ambulance from event 588 has returned to station 43
calling event id: 617 time: 149002 value: 128
calling event id: 618 time: 149004 value: 83
Ambulance for event 590 is returning to the station at 149006
Ambulance from event 585 has returned to station 30
Ambulance for event 598 is returning to the station at 149085
Ambulance has arrived on the scene for event 610 at time 149100
Ambulance has arrived on the scene for event 613 at time 149104
Ambulance has arrived on the scene for event 614 at time 149108
Ambulance from event 601 has returned to station 25
calling event id: 619 time: 149159 value: 82
Ambulance for event 604 has arrived at the hospital at time 149186
Ambul

calling event id: 640 time: 153564 value: 54
Ambulance for event 632 has arrived at the hospital at time 153628
Ambulance from event 621 has returned to station 28
Ambulance has arrived on the scene for event 635 at time 153718
Ambulance for event 624 is returning to the station at 153725
Ambulance for event 634 has arrived at the hospital at time 153744
Ambulance has arrived on the scene for event 637 at time 153792
Ambulance from event 615 has returned to station 26
Ambulance has arrived on the scene for event 640 at time 153864
calling event id: 641 time: 153877 value: 82
Ambulance for event 633 has arrived at the hospital at time 153902
Ambulance for event 631 has arrived at the hospital at time 153977
Ambulance has arrived on the scene for event 638 at time 153978
Ambulance for event 630 is returning to the station at 154027
Ambulance has arrived on the scene for event 639 at time 154059
calling event id: 642 time: 154120 value: 68
calling event id: 643 time: 154169 value: 66
Ambu

Ambulance for event 661 has arrived at the hospital at time 158699
calling event id: 664 time: 158821 value: 99
Ambulance for event 657 has arrived at the hospital at time 158841
calling event id: 665 time: 158884 value: 99
665: call from 99 is a shortfall
Ambulance for event 662 has arrived at the hospital at time 158907
Ambulance for event 656 has arrived at the hospital at time 158970
calling event id: 666 time: 158989 value: 98
calling event id: 667 time: 159034 value: 68
Ambulance for event 658 has arrived at the hospital at time 159140
Ambulance for event 663 has arrived at the hospital at time 159169
Ambulance from event 651 has returned to station 4
calling event id: 665 time: 159177 value: 99
This is a shortfallAmbulance has arrived on the scene for event 667 at time 159334
Ambulance for event 659 is returning to the station at 159349
Ambulance has arrived on the scene for event 665 at time 159477
Ambulance has arrived on the scene for event 664 at time 159578
Ambulance for ev

Excessive output truncated after 524319 bytes.

In [84]:
print(mean(df["responsetime"]))

9.7461

In [51]:
array = filter(x->typeof(x)==gui_event,guiArray)
responded_array = filter(x->x.event_type == "call responded", array)

1000-element Array{Any,1}:
 gui_event("call responded", 83, 1, 1, 1, -1, 0, DateTime("2021-05-08T22:41:55.359"))
 gui_event("call responded", 38, 43, 2, 48, -1, 1, DateTime("2021-05-08T22:41:55.391"))
 gui_event("call responded", 129, 36, 3, 44, -1, 0, DateTime("2021-05-08T22:41:55.407"))
 gui_event("call responded", 107, 34, 4, 42, -1, 1, DateTime("2021-05-08T22:41:55.419"))
 gui_event("call responded", 113, 14, 5, 14, -1, 0, DateTime("2021-05-08T22:41:55.485"))
 gui_event("call responded", 51, 22, 6, 26, -1, 0, DateTime("2021-05-08T22:41:55.516"))
 gui_event("call responded", 76, 39, 7, 45, -1, 1, DateTime("2021-05-08T22:41:55.562"))
 gui_event("call responded", 67, 8, 8, 9, -1, 2, DateTime("2021-05-08T22:41:55.623"))
 gui_event("call responded", 52, 8, 9, 10, -1, 1, DateTime("2021-05-08T22:41:55.623"))
 gui_event("call responded", 67, 8, 10, 11, -1, 0, DateTime("2021-05-08T22:41:55.654"))
 gui_event("call responded", 67, 3, 11, 6, -1, 0, DateTime("2021-05-08T22:41:55.685"))
 gui_eve

In [72]:
mean(df["responsetime"])

9.089033333333333

In [25]:
using DelimitedFiles
filename = string(model_name,string(namb),"calls",string(ncalls),".csv")
open(filename, "w") do filename
    writedlm(filename,  df["responsetime"])
end
print(filename)

Stochastic50calls1000.csv

In [18]:
#using CSV
#filename = string(joshpath,"test/",model_name,string(namb),"calls",string(ncalls),".csv")
#CSV.write(filename, df, delim=",")